In [1]:
import pandas as pd
import wandb
api = wandb.Api()

In [2]:
entity, project = "gregor99", "view_batch_model"
runs = api.runs(entity + "/" + project)

In [3]:

df_selected = pd.DataFrame(columns=['model', 'seed', 'aug_repeat', 'prog_aug', 'distill_type', 'flag', 'RESULT_task_mean_accs', 'RESULT_class_mean_accs', 'TASK_incremental_avg', 'CLASS_incremental_avg', 'BWT_task', 'BWT_class'])
for run in runs:
    if run.config['model'] == 'icarl' and run.config['seed'] in [1993, 1996, 1997] and run.config['dataset'] == 'seq-cifar10' and run.config['buffer_size'] == 200 and run.state == 'finished':
        obj = run.history(keys=['RESULT_task_mean_accs', 'RESULT_class_mean_accs'])
        average_results = obj.mean()
        
        bwt_task = 0
        bwt_class = 0
        num_tasks = 5
        for i in range(num_tasks):
            key_task = 'RESULT_task_acc_' + str(i)
            key_class = 'RESULT_class_acc_' + str(i)
            tmp_result = run.history(keys=[key_task, key_class])

            task_max = tmp_result[key_task].max()
            class_max = tmp_result[key_class].max()

            bwt_task += task_max - tmp_result[key_task].iloc[-1]
            bwt_class += class_max - tmp_result[key_class].iloc[-1]
        bwt_class = bwt_class / (num_tasks - 1)
        bwt_task = bwt_task / (num_tasks - 1)
        
        df_selected.loc[-1] = [run.config['model'], run.config['seed'], run.config['aug_repeat'], run.config.get('prog_aug', 0), run.config.get('distill_type', 2), run.config.get('lr'), run.summary['RESULT_task_mean_accs'], run.summary['RESULT_class_mean_accs'], average_results['RESULT_task_mean_accs'], average_results['RESULT_class_mean_accs'], bwt_task, bwt_class]
        df_selected.index = df_selected.index + 1
        df_selected = df_selected.sort_index()

# df_selected = df_selected[-6:-3]
df_selected

,model,seed,aug_repeat,prog_aug,distill_type,flag,RESULT_task_mean_accs,RESULT_class_mean_accs,TASK_incremental_avg,CLASS_incremental_avg,BWT_task,BWT_class
0,icarl,1997,4,0,2,0.1,92.47,67.34,85.984000,70.566333,0.1500,17.0250
1,icarl,1996,4,0,2,0.1,91.86,67.56,86.683667,71.179500,1.0500,18.6000
2,icarl,1993,4,0,2,0.1,90.55,66.74,88.274167,71.793833,3.6375,20.5375
3,icarl,1993,4,2,2,0.1,92.19,69.59,88.033000,71.201333,3.3625,11.9875
4,icarl,1997,4,5,2,0.1,92.04,67.22,89.598833,73.132333,2.5375,19.2250
5,icarl,1997,1,0,2,0.1,93.94,66.58,90.089667,71.986833,0.8500,23.1875
6,icarl,1993,4,5,2,0.1,90.92,67.87,87.900667,72.076500,3.4375,17.4125
7,icarl,1996,4,2,2,0.1,92.66,69.37,88.058667,72.771500,1.7375,14.8375
8,icarl,1997,4,2,2,0.1,93.33,68.79,88.741833,73.178000,0.7750,14.6125
9,icarl,1996,4,5,2,0.1,90.10,66.38,86.661667,70.899333,2.8625,18.2375


In [4]:
# Calculate the aggregated average
df_grouped = df_selected.groupby(['aug_repeat', 'prog_aug', 'distill_type', 'flag']).agg({'RESULT_task_mean_accs': ['mean', 'std'], 'RESULT_class_mean_accs': ['mean', 'std'], 'TASK_incremental_avg': ['mean', 'std'], 'CLASS_incremental_avg': ['mean', 'std'], 'BWT_task': ['mean', 'std'], 'BWT_class': ['mean', 'std']})

df_grouped['AVG'] = (df_grouped['RESULT_class_mean_accs']['mean'] + df_grouped['RESULT_task_mean_accs']['mean']) / 2
df_grouped['RESULT_task_mean_accs'] = df_grouped['RESULT_task_mean_accs'].apply(lambda x: f'{round(x["mean"], 2)}±{round(x["std"], 2)}', axis=1)
df_grouped['RESULT_class_mean_accs'] = df_grouped['RESULT_class_mean_accs'].apply(lambda x: f'{round(x["mean"], 2)}±{round(x["std"], 2)}', axis=1)
df_grouped['TASK_incremental_avg'] = df_grouped['TASK_incremental_avg'].apply(lambda x: f'{round(x["mean"], 2)}±{round(x["std"], 2)}', axis=1)
df_grouped['CLASS_incremental_avg'] = df_grouped['CLASS_incremental_avg'].apply(lambda x: f'{round(x["mean"], 2)}±{round(x["std"], 2)}', axis=1)
df_grouped['BWT_task'] = df_grouped['BWT_task'].apply(lambda x: f'{round(x["mean"], 2)}±{round(x["std"], 2)}', axis=1)
df_grouped['BWT_class'] = df_grouped['BWT_class'].apply(lambda x: f'{round(x["mean"], 2)}±{round(x["std"], 2)}', axis=1)
df_grouped.columns = df_grouped.columns.droplevel(1)
df_grouped = df_grouped[['RESULT_task_mean_accs', 'RESULT_class_mean_accs', 'AVG', 'TASK_incremental_avg', 'CLASS_incremental_avg', 'BWT_task', 'BWT_class']]
df_grouped = df_grouped.loc[:, ~df_grouped.columns.duplicated()]
df_grouped = df_grouped.reset_index()
df_grouped.rename(columns={'RESULT_task_mean_accs': 'Task-IL', 'RESULT_class_mean_accs': 'Class-IL', 'TASK_incremental_avg': 'Task-IL-Avg', 'CLASS_incremental_avg': 'Class-IL-Avg'}, inplace=True)
df_grouped = df_grouped[['aug_repeat', 'prog_aug', 'distill_type', 'flag', 'Class-IL', 'Task-IL', 'AVG', 'Class-IL-Avg', 'Task-IL-Avg', 'BWT_class', 'BWT_task']]	

# df_grouped.to_excel('tmp.xlsx')
df_grouped

,aug_repeat,prog_aug,distill_type,flag,Class-IL,Task-IL,AVG,Class-IL-Avg,Task-IL-Avg,BWT_class,BWT_task
0,1,0,2,0.1,63.58±2.64,90.32±3.19,76.951667,71.42±0.96,88.65±1.41,28.05±4.21,4.18±2.95
1,1,5,2,0.1,65.33±1.05,89.33±0.58,77.328333,69.47±0.73,85.84±0.68,22.16±0.91,3.61±0.58
2,4,0,2,0.1,67.21±0.42,91.63±0.98,79.420000,71.18±0.61,86.98±1.17,18.72±1.76,1.61±1.81
3,4,2,2,0.1,69.25±0.41,92.73±0.57,80.988333,72.38±1.04,88.28±0.4,13.81±1.58,1.96±1.31
4,4,5,2,0.1,67.16±0.75,91.02±0.97,79.088333,72.04±1.12,88.05±1.47,18.29±0.91,2.95±0.46
